In [1]:
# ! pip install xgboost category_encoders

# ETL

## Settings

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import helpers.settings as sts
sts.print_settings(sts)

BEST_ESTIMATOR_FILENAME : best_estimator_0.0.1.pkl
DATASET_TRAIN_FILENAME : dataset_train.parquet
DATASET_VALIDATION_FILENAME : dataset_validation.parquet
ETL_VERSION : 0.0.1
MODEL_FILENAME : model_0.0.1.pkl
MODEL_VERSION : 0.0.1
PREPROCESSOR_FILENAME : preprocessor_0.0.1.pkl
TRAINED_BEST_ESTIMATOR_FILENAME : trained_best_estimator_0.0.1.pkl
color : <class 'helpers.settings.color'>
print_settings : <function print_settings at 0x7fc9732af5e0>



## Imports

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    classification_report,
    plot_precision_recall_curve,
    precision_recall_curve,
    average_precision_score,
)
from xgboost import XGBClassifier
from category_encoders.woe import WOEEncoder
from sklearn.preprocessing import RobustScaler
import numpy as np

In [6]:
application = pd.read_parquet("data/application.parquet")
credit_record = pd.read_parquet("data/credit_record.parquet")

## Creating label

### Month balance & status

Month balance: 

The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on

Status:

0: 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5: Overdue or bad debts, write-offs for more than 150 days C: paid off that month X: No loan for the month

<font color="red">I will assume that any overdue of 60 or more days corresponds to a default</font>

In [7]:
defaulted_user_ids = credit_record.query("STATUS not in ['0','1','C','X']")["ID"].unique()

In [8]:
credit_record_unique = credit_record.drop_duplicates(subset=["ID"],keep="last",ignore_index=True).copy()

In [9]:
credit_record_unique["LABEL"] = credit_record["ID"].apply(lambda x: 1 if x in defaulted_user_ids else 0)

In [10]:
credit_record_unique["LABEL"].value_counts(normalize=True)

0    0.988583
1    0.011417
Name: LABEL, dtype: float64

### Merge applications with credit records

In [11]:
df = application.merge(credit_record_unique[["ID","LABEL"]], how="inner", on="ID")

### Drop duplicates

In [12]:
df.drop_duplicates(subset=["ID"], keep="first", inplace=True)

In [13]:
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,LABEL
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0


## Split datasets

In [14]:
X = df.iloc[:,1:-1] 
y = df.iloc[:,-1] 

In [15]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=42)

In [16]:
y_train.value_counts(normalize=True)

0    0.986963
1    0.013037
Name: LABEL, dtype: float64

## Dump datasets

In [17]:
pd.concat([X_train, y_train], axis=1).to_parquet(f"data/{sts.DATASET_TRAIN_FILENAME}", index=False)
pd.concat([X_test, y_test], axis=1).to_parquet(f"data/{sts.DATASET_VALIDATION_FILENAME}", index=False)